In [1]:
"""
ЯЧЕЙКА 1: Настройка GPU (ЗАПУСТИТЬ ПЕРВОЙ!)
"""
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # RX 7800 XT
os.environ["HIP_VISIBLE_DEVICES"] = "0"

# Включить experimental Mem Efficient Attention для AMD
os.environ["TORCH_ROCM_AOTRITON_ENABLE_EXPERIMENTAL"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["HIP_VISIBLE_DEVICES"] = "0"

import torch

print("="*70)
print("🔍 КОНФИГУРАЦИЯ GPU")
print("="*70)
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
print(f"Архитектура: {torch.cuda.get_device_properties(0).gcnArchName}")

# Должно быть:
# GPU: AMD Radeon RX 7800 XT
# VRAM: 16.0 GB
# Архитектура: gfx1100

# assert "7800 XT" in torch.cuda.get_device_name(0), "❌ Неправильная GPU!"
# print("✅ Правильная GPU выбрана!")
print("="*70)


🔍 КОНФИГУРАЦИЯ GPU
GPU: AMD Radeon Graphics
VRAM: 16.0 GB
Архитектура: gfx1100


/opt/amdgpu/share/libdrm/amdgpu.ids: No such file or directory
/opt/amdgpu/share/libdrm/amdgpu.ids: No such file or directory


In [2]:
import torch
from PIL import Image
from pathlib import Path
import time
import sys
from pathlib import Path
from transformers import AutoTokenizer, AutoImageProcessor
from transformers.models.lighton_ocr.modeling_lighton_ocr import LightOnOcrForConditionalGeneration
import tqdm as notebook_tqdm


# Добавить корень проекта в PYTHONPATH
PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name != "Team_5_AI_Tutor":
    PROJECT_ROOT = PROJECT_ROOT.parents[1]
sys.path.insert(0, str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / "data"
IMAGES_DIR = PROJECT_ROOT / "notebooks" / "vlm-ingestion-pipeline" / "test_images" / "o-predelnom-mnogomernom-raspredelenii"
IMAGE_PATHS = sorted(IMAGES_DIR.glob("*.png"))
OUTPUT_DIR = DATA_DIR / "test_results"
MODEL_PATH =  PROJECT_ROOT / "models" / "LightOnOCR-1B-1025"

# Проверить путь
if not MODEL_PATH.exists():
    raise FileNotFoundError(f"Модель не найдена: {MODEL_PATH}")

print(f"✅ Путь к модели: {MODEL_PATH}")

# Загрузить компоненты
print(f"\n1️⃣ Загрузка tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(str(MODEL_PATH))
print(f"   ✅ {type(tokenizer)}")


print(f"\n2️⃣ Загрузка image_processor...")
image_processor = AutoImageProcessor.from_pretrained(str(MODEL_PATH))
print(f"   ✅ {type(image_processor)}")

print(f"\n3️⃣ Загрузка модели...")
model = LightOnOcrForConditionalGeneration.from_pretrained(
    str(MODEL_PATH),
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
    attn_implementation="sdpa"
)
model.eval()
print(f"   ✅ Модель на {next(model.parameters()).device}")

print("="*70)
print("🎉 ВСЁ ЗАГРУЖЕНО!")
print("="*70)

/home/gna/anaconda3/envs/lightonocr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Путь к модели: /home/gna/workspase/education/MEPHI/Team_5_AI_Tutor/models/LightOnOCR-1B-1025

1️⃣ Загрузка tokenizer...


The tokenizer you are loading from '/home/gna/workspase/education/MEPHI/Team_5_AI_Tutor/models/LightOnOCR-1B-1025' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
You are using a model of type mistral3 to instantiate a model of type lighton_ocr. This is not supported for all configurations of models and can yield errors.


   ✅ <class 'transformers.models.qwen2.tokenization_qwen2.Qwen2Tokenizer'>

2️⃣ Загрузка image_processor...
   ✅ <class 'transformers.models.pixtral.image_processing_pixtral_fast.PixtralImageProcessorFast'>

3️⃣ Загрузка модели...


Loading weights: 100%|██████████| 533/533 [00:00<00:00, 714.88it/s, Materializing param=model.vision_projection.patch_merger.merging_layer.weight]               
The tied weights mapping and config for this model specifies to tie model.language_model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


   ✅ Модель на cuda:0
🎉 ВСЁ ЗАГРУЖЕНО!


In [3]:
"""
Быстрый тест что модель работает
"""
import torch

print("="*70)
print("🧪 ПРОВЕРКА МОДЕЛИ")
print("="*70)

# Проверить что модель на GPU
device = next(model.parameters()).device
print(f"✅ Device: {device}")
print(f"✅ Dtype: {next(model.parameters()).dtype}")

# Проверить что все компоненты загружены
print(f"\n📦 Компоненты:")
print(f"  ✅ Vision encoder: {model.model.vision_encoder is not None}")
print(f"  ✅ Vision projection: {model.model.vision_projection is not None}")
print(f"  ✅ Language model: {model.model.language_model is not None}")
print(f"  ✅ LM head: {model.lm_head is not None}")

# Проверить память GPU
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1024**3
    reserved = torch.cuda.memory_reserved(0) / 1024**3
    print(f"\n💾 VRAM:")
    print(f"  Allocated: {allocated:.2f} GB")
    print(f"  Reserved: {reserved:.2f} GB")

print(f"\n🎉 Модель готова к использованию!")
print("="*70)


🧪 ПРОВЕРКА МОДЕЛИ
✅ Device: cuda:0
✅ Dtype: torch.bfloat16

📦 Компоненты:
  ✅ Vision encoder: True
  ✅ Vision projection: True
  ✅ Language model: True
  ✅ LM head: True

💾 VRAM:
  Allocated: 2.17 GB
  Reserved: 2.17 GB

🎉 Модель готова к использованию!


In [4]:
"""
ЯЧЕЙКА 3: Тест OCR
"""
import torch
from PIL import Image
from pathlib import Path
import time

print("="*70)
print("📝 ТЕСТ OCR")
print("="*70)


if not IMAGE_PATHS:
    print(f"❌ Изображения не найдены в {IMAGES_DIR}")
    raise FileNotFoundError(f"Нет изображений в {IMAGES_DIR}")

print(f"✅ Найдено изображений: {len(IMAGE_PATHS)}")

# Тест на первом изображении
test_image = IMAGE_PATHS[0]
image = Image.open(test_image).convert("RGB")

print(f"\n📷 Файл: {test_image.name}")
print(f"   Размер: {image.size}")

# Обработать изображение
print(f"\n1️⃣ Обработка изображения...")
image_inputs = image_processor(images=image, return_tensors="pt")
pixel_values = image_inputs['pixel_values'].to("cuda:0", dtype=torch.bfloat16)
print(f"   ✅ pixel_values: {pixel_values.shape}")

# Подготовить текстовый prompt
print(f"\n2️⃣ Подготовка prompt...")
if hasattr(tokenizer, 'apply_chat_template'):
    messages = [{"role": "user", "content": [{"type": "image"}]}]
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    print(f"   ✅ Chat template: {prompt[:50]}...")
else:
    # Fallback: простой prompt
    prompt = ""
    print(f"   ⚠️  Chat template недоступен, используем пустой prompt")

# Токенизировать
text_inputs = tokenizer(
    prompt,
    return_tensors="pt",
    add_special_tokens=True
).to("cuda:0")
print(f"   ✅ input_ids: {text_inputs['input_ids'].shape}")

# Генерация
print(f"\n3️⃣ Генерация текста...")
start_time = time.time()

with torch.no_grad():
    outputs = model.generate(
        input_ids=text_inputs['input_ids'],
        pixel_values=pixel_values,
        max_new_tokens=2000,
        temperature=0.2,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

generation_time = time.time() - start_time
print(f"   ✅ Завершено за {generation_time:.2f}с")

# Декодирование
print(f"\n4️⃣ Декодирование...")
input_length = text_inputs['input_ids'].shape[1]
generated_text = tokenizer.decode(
    outputs[0, input_length:],
    skip_special_tokens=True
)

# Результат
print(f"\n{'='*70}")
print(f"📝 РЕЗУЛЬТАТ OCR")
print(f"{'='*70}")
print(f"⏱️  Время: {generation_time:.2f}с")
print(f"📊 Символов: {len(generated_text)}")
print(f"📊 Токенов: {outputs.shape[1] - input_length}")

if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1024**3
    print(f"💾 VRAM: {allocated:.2f} GB")

print(f"\n📄 ТЕКСТ:")
print(generated_text)
print(f"{'='*70}")

if len(generated_text) > 100:
    print(f"\n🎉 УСПЕХ! LightOnOCR работает на RX 7800 XT!")
else:
    print(f"\n⚠️  Результат короткий ({len(generated_text)} символов)")


📝 ТЕСТ OCR
✅ Найдено изображений: 4

📷 Файл: page_001.png
   Размер: (1211, 1713)

1️⃣ Обработка изображения...
   ✅ pixel_values: torch.Size([1, 3, 1540, 1092])

2️⃣ Подготовка prompt...
   ✅ Chat template: <|im_start|>system<|im_end|>
<|im_start|>user
<|im...
   ✅ input_ids: torch.Size([1, 13])

3️⃣ Генерация текста...


TypeError: 'NoneType' object is not iterable

In [7]:
"""
ЯЧЕЙКА 3 ФИНАЛЬНАЯ: OCR с Правильным Prompt
"""
import torch
from PIL import Image
from pathlib import Path
import time

print("="*70)
print("📝 ТЕСТ OCR (ФИНАЛЬНАЯ ВЕРСИЯ)")
print("="*70)

# Загрузить изображение
test_image = IMAGE_PATHS[0]
image = Image.open(test_image).convert("RGB")

print(f"\n📷 Файл: {test_image.name}")
print(f"   Исходный размер: {image.size}")

# Resize
max_size = 800
if max(image.size) > max_size:
    ratio = max_size / max(image.size)
    new_size = (int(image.size[0] * ratio), int(image.size[1] * ratio))
    image = image.resize(new_size, Image.Resampling.LANCZOS)
    print(f"   ✅ Resized до: {image.size}")

# Обработать изображение
print(f"\n1️⃣ Обработка изображения...")
image_inputs = image_processor(images=image, return_tensors="pt")
pixel_values = image_inputs['pixel_values'].to("cuda:0", dtype=torch.bfloat16)
image_sizes = torch.tensor([[image.height, image.width]], device="cuda:0")

print(f"   ✅ pixel_values: {pixel_values.shape}")
print(f"   ✅ image_sizes: {image_sizes}")

# Вычислить количество image tokens
# Формула: (height // patch_size) * (width // patch_size) // spatial_merge_size^2
patch_size = model.config.vision_config.patch_size  # обычно 14
spatial_merge = model.config.spatial_merge_size  # обычно 2

h_patches = image.height // patch_size
w_patches = image.width // patch_size
n_image_tokens = (h_patches * w_patches) // (spatial_merge * spatial_merge)

print(f"\n   📊 Patches: {h_patches}×{w_patches} = {h_patches * w_patches}")
print(f"   📊 После merge: {n_image_tokens} tokens")

# Создать prompt с правильным количеством image tokens
print(f"\n2️⃣ Подготовка prompt...")

# Получить image token
image_token = "<|image|>"
image_token_id = tokenizer.convert_tokens_to_ids(image_token)

if image_token_id == tokenizer.unk_token_id:
    # Fallback: использовать специальный токен
    print(f"   ⚠️  Image token не найден, используем альтернативный подход")
    # Создать prompt без image tokens (модель сама вставит)
    prompt = ""
    input_ids = tokenizer("", return_tensors="pt", add_special_tokens=False)["input_ids"].to("cuda:0")
else:
    # Создать prompt с N image tokens
    prompt = image_token * n_image_tokens
    print(f"   ✅ Prompt: {len(prompt)} символов ({n_image_tokens} image tokens)")
    
    input_ids = tokenizer(
        prompt,
        return_tensors="pt",
        add_special_tokens=False
    )["input_ids"].to("cuda:0")

print(f"   ✅ input_ids: {input_ids.shape}")

# Генерация
print(f"\n3️⃣ Генерация текста...")
print(f"   (Ожидайте ~30-60 секунд...)")
start_time = time.time()

try:
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            pixel_values=pixel_values,
            image_sizes=image_sizes,
            max_new_tokens=512,
            temperature=0.2,
            top_p=0.9,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    generation_time = time.time() - start_time
    print(f"   ✅ Завершено за {generation_time:.2f}с")
    
    # Декодирование
    input_length = input_ids.shape[1]
    generated_text = tokenizer.decode(
        outputs[0, input_length:],
        skip_special_tokens=True
    )
    
    # Результат
    print(f"\n{'='*70}")
    print(f"📝 РЕЗУЛЬТАТ OCR")
    print(f"{'='*70}")
    print(f"⏱️  Время: {generation_time:.2f}с")
    print(f"📊 Символов: {len(generated_text)}")
    print(f"📊 Токенов: {outputs.shape[1] - input_length}")
    print(f"💾 VRAM: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    
    print(f"\n📄 ТЕКСТ:")
    print(generated_text)
    print(f"{'='*70}")
    
    if len(generated_text) > 50:
        print(f"\n🎉 УСПЕХ! LightOnOCR работает")
    else:
        print(f"\n⚠️  Результат короткий")

except Exception as e:
    print(f"\n❌ ОШИБКА: {e}")
    import traceback
    traceback.print_exc()


📝 ТЕСТ OCR (ФИНАЛЬНАЯ ВЕРСИЯ)

📷 Файл: page_001.png
   Исходный размер: (1211, 1713)
   ✅ Resized до: (565, 800)

1️⃣ Обработка изображения...
   ✅ pixel_values: torch.Size([1, 3, 812, 574])
   ✅ image_sizes: tensor([[800, 565]], device='cuda:0')

   📊 Patches: 57×40 = 2280
   📊 После merge: 570 tokens

2️⃣ Подготовка prompt...
   ⚠️  Image token не найден, используем альтернативный подход
   ✅ input_ids: torch.Size([1, 0])

3️⃣ Генерация текста...
   (Ожидайте ~30-60 секунд...)

❌ ОШИБКА: index -1 is out of bounds for dimension 0 with size 0


Traceback (most recent call last):
  File "/tmp/ipykernel_1145959/112502302.py", line 82, in <module>
    outputs = model.generate(
  File "/home/gna/anaconda3/envs/lightonocr/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 120, in decorate_context
    return func(*args, **kwargs)
  File "/home/gna/anaconda3/envs/lightonocr/lib/python3.10/site-packages/transformers/generation/utils.py", line 2684, in generate
    result = decoding_method(
  File "/home/gna/anaconda3/envs/lightonocr/lib/python3.10/site-packages/transformers/generation/utils.py", line 2877, in _sample
    outputs = self._prefill(input_ids, generation_config, model_kwargs)
  File "/home/gna/anaconda3/envs/lightonocr/lib/python3.10/site-packages/transformers/generation/utils.py", line 3852, in _prefill
    model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)
  File "/home/gna/anaconda3/envs/lightonocr/lib/python3.10/site-packages/transformers/models/lighton_ocr/modeling_lighton_ocr.py

In [11]:
"""
Альтернативный подход через Processor
"""
from transformers.models.lighton_ocr.processing_lighton_ocr import LightOnOcrProcessor

# Попробовать загрузить полный processor
try:
    processor = LightOnOcrProcessor.from_pretrained(str(MODEL_PATH))
    print(f"✅ LightOnOcrProcessor загружен")
    
    # Использовать processor для подготовки inputs
    inputs = processor(
        images=image,
        return_tensors="pt"
    ).to("cuda:0")
    
    inputs["pixel_values"] = inputs["pixel_values"].to(torch.bfloat16)
    
    # Генерация
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.2,
            do_sample=False,
        )
    
    # Декодирование
    text = processor.decode(outputs[0], skip_special_tokens=True)
    print(f"📝 ТЕКСТ: {text}")
    
except Exception as e:
    print(f"❌ LightOnOcrProcessor недоступен: {e}")


❌ LightOnOcrProcessor недоступен: transformers.tokenization_utils_tokenizers.TokenizersBackend._patch_mistral_regex() got multiple values for keyword argument 'fix_mistral_regex'


In [12]:
"""
Проверка конфигурации модели
"""
import json

config_path = MODEL_PATH / "config.json"
with open(config_path, "r") as f:
    config = json.load(f)

print("📋 Конфигурация LightOnOCR:")
print(f"  image_token_id: {config.get('image_token_id', 'НЕ НАЙДЕН')}")
print(f"  patch_size: {config['vision_config']['patch_size']}")
print(f"  spatial_merge_size: {config.get('spatial_merge_size', 'НЕ НАЙДЕН')}")

# Проверить tokenizer
print(f"\n📋 Специальные токены:")
print(f"  vocab_size: {tokenizer.vocab_size}")
print(f"  eos_token: {tokenizer.eos_token} (id: {tokenizer.eos_token_id})")
print(f"  pad_token: {tokenizer.pad_token} (id: {tokenizer.pad_token_id})")
print(f"  bos_token: {tokenizer.bos_token} (id: {tokenizer.bos_token_id})")

# Найти image token в vocab
vocab = tokenizer.get_vocab()
image_tokens = [k for k in vocab.keys() if 'image' in k.lower()]
print(f"\n📋 Image-related tokens: {image_tokens[:10]}")


📋 Конфигурация LightOnOCR:
  image_token_id: 151655
  patch_size: 14
  spatial_merge_size: 2

📋 Специальные токены:
  vocab_size: 151643
  eos_token: <|im_end|> (id: 151645)
  pad_token: <|endoftext|> (id: 151643)
  bos_token: None (id: None)

📋 Image-related tokens: ['.BackgroundImage', 'ĠsetBackgroundImage', '.ImageIcon', 'imageUrl', '.StretchImage', 'ĠloadImage', 'Ġimagen', '_images', 'ImageUrl', ',image']


In [13]:
"""
ЯЧЕЙКА: РАБОЧИЙ OCR С ПРАВИЛЬНЫМ image_token_id
"""
import torch
from PIL import Image
import time

print("="*70)
print("🚀 LIGHTONOCR - ФИНАЛЬНЫЙ ТЕСТ")
print("="*70)

# Загрузить изображение
test_image = IMAGE_PATHS[0]
image = Image.open(test_image).convert("RGB")

print(f"\n📷 Файл: {test_image.name}")
print(f"   Размер: {image.size}")

# Resize
max_size = 768
if max(image.size) > max_size:
    ratio = max_size / max(image.size)
    new_size = (int(image.size[0] * ratio), int(image.size[1] * ratio))
    image = image.resize(new_size, Image.Resampling.LANCZOS)
    print(f"   ✅ Resized: {image.size}")

# Обработка изображения
print(f"\n1️⃣ Обработка изображения...")
image_inputs = image_processor(images=image, return_tensors="pt")
pixel_values = image_inputs['pixel_values'].to("cuda:0", dtype=torch.bfloat16)

# image_sizes как LIST (не tensor!)
image_sizes = [[image.height, image.width]]

print(f"   ✅ pixel_values: {pixel_values.shape}")
print(f"   ✅ image_sizes: {image_sizes}")

# Вычислить количество image tokens
patch_size = 14
spatial_merge = 2

h_patches = image.height // patch_size
w_patches = image.width // patch_size
n_image_tokens = (h_patches * w_patches) // (spatial_merge ** 2)

print(f"   📊 Patches: {h_patches}×{w_patches}")
print(f"   📊 Image tokens: {n_image_tokens}")

# Создать input_ids с правильным image_token_id
print(f"\n2️⃣ Создание input_ids...")

# Использовать image_token_id из config
image_token_id = 151655  # Из вашего config

# ПРОВЕРКА: убедиться что модель знает этот токен
if image_token_id >= len(tokenizer):
    print(f"   ⚠️  image_token_id ({image_token_id}) > vocab_size ({len(tokenizer)})")
    print(f"   ✅ Расширяем embeddings...")
    
    # Расширить embeddings модели
    model.resize_token_embeddings(image_token_id + 1)
    print(f"   ✅ Новый размер: {len(tokenizer)} → {image_token_id + 1}")

# Создать input_ids
input_ids = torch.full(
    (1, n_image_tokens),
    image_token_id,
    dtype=torch.long,
    device="cuda:0"
)

print(f"   ✅ input_ids: {input_ids.shape}")
print(f"   ✅ image_token_id: {image_token_id}")

# Генерация
print(f"\n3️⃣ Генерация текста...")
print(f"   (Ожидание ~30-60 секунд на RX 7800 XT...)")

start_time = time.time()

try:
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            pixel_values=pixel_values,
            image_sizes=image_sizes,  # LIST!
            max_new_tokens=1024,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    generation_time = time.time() - start_time
    print(f"   ✅ Завершено за {generation_time:.2f}с")
    
    # Декодирование
    input_length = input_ids.shape[1]
    generated_text = tokenizer.decode(
        outputs[0, input_length:],
        skip_special_tokens=True
    )
    
    # Результат
    print(f"\n{'='*70}")
    print(f"✨ РЕЗУЛЬТАТ OCR ✨")
    print(f"{'='*70}")
    print(f"⏱️  Время: {generation_time:.2f}с")
    print(f"📊 Символов: {len(generated_text)}")
    print(f"📊 Токенов: {outputs.shape[1] - input_length}")
    print(f"⚡ Скорость: {(outputs.shape[1] - input_length) / generation_time:.1f} tok/s")
    print(f"💾 VRAM: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    
    print(f"\n📄 РАСПОЗНАННЫЙ ТЕКСТ:")
    print(f"{'='*70}")
    print(generated_text[:2000])  # Первые 2000 символов
    if len(generated_text) > 2000:
        print(f"\n... (ещё {len(generated_text) - 2000} символов)")
    print(f"{'='*70}")
    
    if len(generated_text) > 50:
        print(f"\n🎉 🎉 🎉 УСПЕХ! 🎉 🎉 🎉")
        print(f"")
        print(f"✅ LightOnOCR работает на AMD RX 7800 XT!")
        print(f"✅ ROCm + PyTorch успешно справились!")
        print(f"")
        print(f"📊 Статистика:")
        print(f"   • Время: {generation_time:.2f}с")
        print(f"   • Скорость: {(outputs.shape[1] - input_length) / generation_time:.1f} токенов/с")
        print(f"   • VRAM: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB / 16 GB")
        
        # Сохранить результат
        output_file = Path("ocr_result.txt")
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(f"Файл: {test_image.name}\n")
            f.write(f"Время: {generation_time:.2f}с\n")
            f.write(f"Символов: {len(generated_text)}\n")
            f.write(f"\n{'='*70}\n")
            f.write(generated_text)
        print(f"\n💾 Результат сохранён в: {output_file}")
        
    else:
        print(f"\n⚠️  Результат короткий ({len(generated_text)} символов)")

except Exception as e:
    print(f"\n❌ ОШИБКА: {e}")
    import traceback
    traceback.print_exc()
    
    print(f"\n🔍 Debug info:")
    print(f"   input_ids: {input_ids.shape}")
    print(f"   pixel_values: {pixel_values.shape}")
    print(f"   image_sizes: {image_sizes}")


🚀 LIGHTONOCR - ФИНАЛЬНЫЙ ТЕСТ

📷 Файл: page_001.png
   Размер: (1211, 1713)
   ✅ Resized: (542, 768)

1️⃣ Обработка изображения...
   ✅ pixel_values: torch.Size([1, 3, 770, 546])
   ✅ image_sizes: [[768, 542]]
   📊 Patches: 54×38
   📊 Image tokens: 513

2️⃣ Создание input_ids...
   ✅ input_ids: torch.Size([1, 513])
   ✅ image_token_id: 151655

3️⃣ Генерация текста...
   (Ожидание ~30-60 секунд на RX 7800 XT...)
   ✅ Завершено за 17.75с

✨ РЕЗУЛЬТАТ OCR ✨
⏱️  Время: 17.75с
📊 Символов: 1644
📊 Токенов: 1024
⚡ Скорость: 57.7 tok/s
💾 VRAM: 2.20 GB

📄 РАСПОЗНАННЫЙ ТЕКСТ:
raz*****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************